In [ ]:
from functools import reduce
import re
import csv
from tqdm import tqdm
from BTrees.OOBTree import OOBTree

In [ ]:
# Lista dei postings aka i docID
class PostingsList:

    def __init__(self) -> None:
        self._postings_list = []

    # crea una PostingsList da una lista di docID e la ordina (forse non necssario)
    @classmethod
    def from_postings_list(cls, postings_list: list[int]) -> 'PostingsList':
        plist = cls()
        postings_list.sort()
        plist._postings_list = postings_list
        return plist

    # Crea una PostingsList da un singolo docID
    @classmethod
    def from_doc_id(cls, doc_id: int) -> 'PostingsList':
        plist = cls()
        plist._postings_list = [doc_id]
        return plist

    # Concatena due PostingsList. Le liste sono ordinate e i duplicati rimossi. other contiene una PostingsList creata successivamente a self (i docID saranno piu grandi o uguali)
    def merge(self, other: "PostingsList") -> 'PostingsList':
        if self._postings_list == []:
            self._postings_list = other._postings_list
        i = 0  # Start index for the other PostingList.
        last = self._postings_list[-1]  # The last Posting in the current list.
        # Loop through the other PostingList and skip duplicates.
        while (i < len(other._postings_list) and last == other._postings_list[i]):
            i += 1  # Increment the index if a duplicate is found.
        # Append the non-duplicate postings from the other list.
        self._postings_list += other._postings_list[i:]
        return self

    # Ottiene i titoli di documenti dai docID nella PostingsList
    def get_from_corpus(self, corpus) -> list[str]:
        return list(map(lambda x: corpus[x], self._postings_list))

    # Effettua l'intersezione di due PostgingsList con il metodo del doppio indice
    def intersection(self, other: "PostingsList") -> 'PostingsList':
        plist = []
        i = 0  # indice riferito a self
        j = 0  # indice riferito a other
        # finché non si eccede la dimensione di ciascuna lista:
        while (i < len(self._postings_list)) and (j < len(other._postings_list)):
            # se c'e' un match aggiungi l'elemento e incrmeneta entrambi
            if self._postings_list[i] == other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
                j += 1
            # altrimenti aumenta il piu piccolo dei due
            elif self._postings_list[i] <= other._postings_list[j]:
                i += 1
            # altrimenti aumenta l'altro
            else:
                j += 1
        return PostingsList.from_postings_list(plist)

    # Effettua l'unione di due PostingsList con il metodo del doppio indice
    def union(self, other: "PostingsList") -> 'PostingsList':
        plist = []
        i = 0  # indice riferito a self
        j = 0  # indice riferito a other
        # fintanto che gli indici sono piu' piccoli di entrambe le liste
        while (i < len(self._postings_list)) and (j < len(other._postings_list)):
            # aggiungi il docID e aumenta entrambi
            if self._postings_list[i] == other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
                j += 1
            # altrimenti aggiungi il docID e aumenta il piu' piccolo
            elif self._postings_list[i] < other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
            #  aggiungi l'altro e incrementalo
            else:
                plist.append(other._postings_list[j])
                j += 1
        # aggiungi la porzione restante di lista
        if i < len(self._postings_list):  # nel caso in cui self era piu' lunga
            plist += self._postings_list[i:]
        elif j < len(other._postings_list):  # nel caso in cui other era piu' lunga
            plist += other._postings_list[j:]
        return PostingsList.from_postings_list(plist)

    # Effettua la negazione del tipo AND NOT con il metodo dei due indici
    def negation(self, other: 'PostingsList') -> 'PostingsList':
        plist = []
        i = 0
        j = 0
        while (i < len(self._postings_list)) and (j < len(other._postings_list)):
            # se self contiene il docID, scartalo e incrementa entrambi
            if self._postings_list[i] == other._postings_list[j]:
                i += 1
                j += 1
            # aggiungi il docID da self e incrementa se e' piu' piccolo
            elif self._postings_list[i] < other._postings_list[j]:
                plist.append(self._postings_list[i])
                i += 1
            # incrementa other
            else:
                j += 1
        # aggiungi i documenti mancanti da self
        if i < len(self._postings_list):  # se e' piu' lungo di other
            plist += self._postings_list[i:]
        return PostingsList.from_postings_list(plist)

    def __repr__(self) -> str:
        return ", ".join(map(str, self._postings_list))

In [ ]:
def normalize(text):
    """Remove punctuation and convert text to lowercase"""
    return re.sub(r'[^\w\s^-]', '', text).lower()


def tokenize(content) -> list:
    """Split normalized text into tokens"""
    return normalize(content).split()


class InvertedIndex:

    def __init__(self) -> None:
        self.btree = OOBTree()  # usa un Btree per rendere piu' veloci aggiornamenti dell'indice

    @classmethod
    def from_corpus(cls, corpus, max_size=0) -> 'InvertedIndex':
        terms = {}  # dizionario temporaneo per tenere l'indice iniziale
        # per ogni documento
        for doc_id, content in enumerate(tqdm(corpus, total=max_size or None)):
            # crea un set dei termini che contiene
            tokens = set(tokenize(content.description))
            for token in tokens:  # per ogni termine
                plist = PostingsList.from_doc_id(doc_id)
                if token in terms:  # se contenuto
                    terms[token].merge(plist)  # fai merge delle PostingsList
                else:  # altrimenti aggiungi
                    terms[token] = plist
        idx = cls()
        idx.btree.update(terms)
        return idx

    # crea il biword index per le phrase queries
    @classmethod
    def from_corpus_biword(cls, corpus, max_size=0) -> 'InvertedIndex':
        terms = {}
        # per ogni documento
        for doc_id, content in enumerate(tqdm(corpus, total=max_size or None)):
            tokens = tokenize(content.description)
            # per ogni parola
            for i in range(len(tokens) - 1):
                biword = tokens[i]+tokens[i+1]
                plist = PostingsList.from_doc_id(doc_id)
                if biword in terms:
                    terms[biword].merge(plist)
                else:
                    terms[biword] = plist
        idx = cls()
        idx.btree.update(terms)
        return idx

    def merge(self, other: 'InvertedIndex') -> 'InvertedIndex':
        for term, postings in other.btree.items():
            if term in self.btree:
                self.btree[term].merge(postings)
            else:
                self.btree[term] = postings
        return self

    def __getitem__(self, key: str) -> PostingsList:
        return self.btree[key]

    def __len__(self) -> int:
        return len(self.btree)

    def __repr__(self) -> str:
        return self.btree

In [ ]:
class MovieDescription:
    def __init__(self, title: str, description: str) -> None:
        self.title = title
        self.description = description

    def __repr__(self) -> str:
        return self.title

# leggi il file descrizione e metadata e crea un corpus (collection di documenti)


def read_movie_description(movie_metadata, description_file) -> list[MovieDescription]:
    names = {}
    corpus = []
    with open(movie_metadata, 'r') as file:  # leggi i metadati
        movie_names = csv.reader(file, delimiter='\t')
        for description in movie_names:  # aggiungi a names la coppia id_film: titolo
            names[description[0]] = description[2]
    with open(description_file, 'r') as file:  # leggi le descrizioni
        descriptions = csv.reader(file, delimiter='\t')
        for description in descriptions:
            try:
                # aggiungi al corpus il titolo e la descrizione di ciascun film
                corpus.append(MovieDescription(
                    # il docID e' la posizione del documento nel corpus
                    names[description[0]], description[1]))
            except KeyError:
                pass
    return corpus

In [ ]:
class IrSystem:
    def __init__(self, corpus: list[MovieDescription], index: InvertedIndex, biword: InvertedIndex, max_size_aux=10000) -> None:
        self._corpus = corpus
        self._index = index  # inverted index
        self._invalid_vec = []  # invalidation bit vector
        self._temp_idx = None  # indice ausiliario
        self.max_size_aux = max_size_aux  # massimo docID assegnato
        self._biword = biword  # inverted index con biword per phrase queries
        self._temp_biword = None

    # Crea l'indice e il biword
    @classmethod
    def from_corpus(cls, corpus: list[MovieDescription]) -> 'IrSystem':
        index = InvertedIndex.from_corpus(corpus)
        biword = InvertedIndex.from_corpus_biword(corpus)
        ir = cls(corpus, index, biword)
        ir._invalid_vec = [0] * len(corpus)
        return ir

    # Segna documenti cancellati
    def delete_docs(self, documents: list[int]) -> 'IrSystem':
        for doc in documents:
            self._invalid_vec[doc] = 1
        return self

    # Aggiungi documenti nuovi all'indice ausiliario
    def add_docs(self, corpus: list[MovieDescription]) -> 'IrSystem':
        # i nuovi documenti usano docID piu' grandi
        aux = InvertedIndex.from_corpus(corpus, len(self._invalid_vec))
        if self._temp_idx is None:  # se non e' presente nell'indice ausiliario
            self._temp_idx = aux  # aggiungilo
        else:  # altrimenti
            self._temp_idx.merge(aux)
        if len(self._temp_idx) > self.max_size_aux:  # se l'indice ausiliario e' troppo grande
            self.merge_idx()  # fai merge

        aux_biword = InvertedIndex.from_corpus_biword(corpus, len(self.invali))
        if self._temp_biword is None:
            self._temp_biword = aux_biword
        else:
            self._temp_biword.merge(aux_biword)
        # aggiorna la dimensione massima attuale
        self.max_size_aux += len(corpus)
        # aggiorna l'invalidation bit vector
        self._invalid_vec += [0] * len(corpus)
        return self

    # Merge dell'indice ausilario con l'InvertedIndex
    def merge_idx(self) -> 'IrSystem':
        self._index.merge(self._temp_idx)
        self._biword.merge(self._temp_biword)
        self._temp_idx = None
        self._temp_biword = None
        return self

    # Effettua una query booleana combinando i termini con AND, OR e NOT
    def query(self, query: str) -> list[str]:
        tokens = query.split()
        # riscrive la query con gli operatori postfix
        postfix = infix_to_postfix(tokens)
        stack = []  # PostingsList ancora da processare
        for token in postfix:
            if token in ('AND', 'OR', 'NOT'):
                right = stack.pop()
                left = stack.pop()
                if token == 'AND':  # caso AND, conviene ottimizzare la query facendo l'intersezione delle liste piu' corte in primis
                    if not isinstance(left, list):
                        left = [left]
                    if not isinstance(right, list):
                        right = [right]
                    # aggiungi allo stack una lista [left, right]
                    stack.append(left + right)
                elif token in ('OR', 'NOT'):
                    if isinstance(left, list):  # se left e' una lista (catena di AND)
                        # effettua la sequenza di AND
                        left = self._optimize_and_query(left)
                    if isinstance(right, list):  # se right e' una lista (catena di AND)
                        # effettua la sequenza di AND
                        right = self._optimize_and_query(right)
                    if token == 'OR':  # effettua l'OR
                        stack.append(left.union(right))
                    else:  # effettua il NOT (AND NOT)
                        stack.append(left.negation(right))
            else:  # aggiungi una PostingsList da processare allo stack
                base = self._index.btree.get(
                    token, PostingsList.from_postings_list([]))
                aux = self._temp_idx.btree.get(token, PostingsList.from_postings_list(
                    [])) if self._temp_idx else PostingsList.from_postings_list([])
                stack.append(base.merge(aux))
        result = stack.pop()  # estrai l'ultimo elemento (risultato finale)
        if isinstance(result, list):  # se e' tuttora una lista (= catena di AND), fai l'intersezione
            result = self._optimize_and_query(result)
        # elimina i documenti cancellati
        return self._remove_deleted(result).get_from_corpus(self._corpus)

    def _remove_deleted(self, result: PostingsList) -> PostingsList:
        for doc_id, deleted in enumerate(self._invalid_vec):
            if deleted and doc_id in result._postings_list:
                result._postings_list.remove(doc_id)
        return result

    # Effettua operazioni di AND consecutive facendo l'intersezione di PostingsList piu' corte prima
    def _optimize_and_query(self, terms: list[PostingsList]) -> PostingsList:
        # ordina le PostingsList per lunghezza crescente
        plist = sorted(terms, key=lambda x: len(x._postings_list))
        result = reduce(lambda x, y: x.intersection(y), plist)
        return result

    # Ricerca una sequenza specifica di parola nel corpus con biword
    def phrase_query(self, query: str) -> list[str]:
        biword_query = []
        words = query.split()
        for i in range(len(words)-1):
            # concatena le parole della query in coppie
            biword_query.append(words[i]+words[i+1])
        postings = []
        # cerca le biword nel biword index
        for biword in biword_query:
            base = self._biword.btree.get(
                biword, PostingsList.from_postings_list([]))
            aux = self._temp_biword.btree.get(biword, PostingsList.from_postings_list(
                [])) if self._temp_biword else PostingsList.from_postings_list([])
            postings.append(base.merge(aux))
        # effettua l'intersezione delle PostingsList trovate
        plist = reduce(lambda x, y: x.intersection(y), postings)
        # rimuovi cancellati e ritorna i risultati
        return self._remove_deleted(plist).get_from_corpus(self._corpus)

# Rende una espressione da infix a postfix: a AND b OR c -> a b AND c OR


def infix_to_postfix(tokens: list[str]) -> list[str]:
    output = []  # risultato finale
    stack = []  # ancora da processare
    for token in tokens:
        if token in ('AND', 'OR', 'NOT'):  # se e' un operatore
            # finche' ci sono parole da processare e non e' una parentesi
            while (stack and stack[-1] != '('):
                # aggiungi al risultato finale le parole una dopo l'altra
                output.append(stack.pop())
            stack.append(token)  # aggiungi l'operatore allo stack
        elif token == '(':  # aggiungi la parentesi allo stack
            stack.append(token)
        elif token == ')':
            # fino a che non incontro la parentesi aperta o si svuota lo stack
            while stack and stack[-1] != '(':
                # aggiungo all'output il contenuto dello stack
                output.append(stack.pop())
            stack.pop()  # remove '('
        else:  # aggiungi un termine all'outpuit
            output.append(token)
    while stack:  # svuota lo stack
        output.append(stack.pop())
    return output

In [67]:
corpus = read_movie_description(
    '../Code IR/data/movie.metadata.tsv', '../Code IR/data/plot_summaries.txt')

In [71]:
ir = IrSystem.from_corpus(corpus)

  0%|          | 0/42204 [00:00<?, ?it/s]

100%|██████████| 42204/42204 [00:35<00:00, 1172.88it/s]


In [72]:
ir.phrase_query('speak during meetings')

[Lord of the Flies]

In [73]:
print(len(ir.query('yoda')), len(ir.query(
    'luke')), len(ir.query('wars')))

13 161 179


In [74]:
ir.query('luke')

[Afghan Luke,
 Daisy Town,
 Decoys 2: Alien Seduction,
 Out Cold,
 2:37,
 Lilies of the Field,
 Scumbus,
 Death of a Gunfighter,
 Fatty and Mabel Adrift,
 Santa Baby,
 The Boys Club,
 SpaceCamp,
 Undiscovered,
 Fast Five,
 Star Wars Episode V: The Empire Strikes Back,
 Dual,
 Angels and Demons,
 Children of Men,
 Spiderhole,
 Spike and Suzy: The Texas Rangers,
 Children of the Corn V: Fields of Terror,
 Stagecoach,
 Animal Kingdom,
 The Prince of Tides,
 The Dukes of Hazzard: Reunion!,
 Vanishing on 7th Street,
 Green Light,
 Still Crazy,
 Coming Home,
 Decoys,
 Halloween Resurrection,
 Imaginationland Episode II,
 Slaves,
 Jennifer,
 Nagarangalil Chennu Raparkam,
 Star Wars Episode IV: A New Hope,
 Memphis Belle,
 Wishology,
 The Wendell Baker Story,
 The Little Troll Prince: A Christmas Parable,
 Mustang Country,
 Macon County Line,
 The Long Kiss Goodnight,
 The Dukes of Hazzard: Hazzard in Hollywood!,
 A Woman's Secret,
 No Name on the Bullet,
 Tanner on Tanner,
 The Toy that Saved

In [77]:
ir.query('afghanistan')

[Getting Even,
 Agent Vinod,
 Afghan Luke,
 Brothers,
 Charlie Wilson's War,
 The Veteran,
 Afghan Breakdown,
 Iron Man,
 Summer Heat,
 New Year's Eve,
 The Minion,
 The Storm,
 The Objective,
 Zombie Strippers,
 Outlaw,
 If I Should Fall,
 Dharmatma,
 The Christmas Card,
 The Boy Mir,
 The Hard Corps,
 Main Osama,
 The Whistleblower,
 Where in the World is Osama Bin Laden?,
 All Costs Paid,
 Kabul Express,
 16 Days in Afghanistan,
 Armadillo,
 Fire Creek,
 Kabuliwala,
 Keerthi Chakra,
 Rambo,
 Aegan,
 9th Company,
 Netaji Subhas Chandra Bose: The Forgotten Hero,
 Qayamat - A Love Triangle In Afghanistan,
 The Kite Runner,
 Beyond the Call,
 Stealing a Nation,
 Lions for Lambs,
 Afghan Massacre - the Convoy of Death,
 Afghan Muscles,
 Rambo III,
 Homeland Security,
 Kim,
 Savages,
 My Name is Khan,
 The Great Mouse Detective,
 Watchmen,
 Brothers,
 DC 9/11: Time of Crisis,
 Khamosh Pani,
 Fahrenheit 9/11,
 La Linea,
 Baran,
 Mission Istanbul,
 Enchantment,
 Osama,
 25 Hill,
 The best M